In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from scipy.spatial.distance import cosine
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import sys

In [ ]:
# extract a single face from an uploaded photograph
detector = MTCNN()
def extract_face(filename,detector=detector,required_size=(224, 224)):
	pixels = plt.imread(filename)
	# create the detector, using default weights
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
    # resize pixels to the model size
  
# create a vggface model
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

#the resnet model represents the face in a 2048-dimension feature space.

def get_embedding(filename,model=model):
    # extract faces
    face = extract_face(filename)
    # convert into an array of samples
    sample = [asarray(face, 'float32')]
    # prepare the face for the model, e.g. center pixels
    sample = preprocess_input(sample, version=2)
    # perform prediction
    yhat = model.predict(sample)
    return yhat

def is_match(ID_embedding, subject_embedding, show_faces,thresh=0.4):
    # calculate distance between embeddings
    score = cosine(ID_embedding, subject_embedding)
    res = {'verified' : score <= 0.4, 'distance': score}

def face_classification(img1, img2):
  embedding1 = get_embedding(img1)
  embedding2 = get_embedding(img2)
  return is_match(embedding1,embedding2)

In [2]:
folderPath = 'drive/MyDrive/FGNET/images'

In [3]:
import os
os.path.exists(folderPath)

True

In [4]:
import pandas as pd

In [6]:
main_dic = dict()
folders = os.listdir(folderPath)
folders.sort()
for folder in folders:
  files = os.listdir(f"{folderPath}/{folder}")
  files.sort()
  image1 = f"{folderPath}/{folder}/{files[2]}"

  print(f"Person {folder}")

  df_dict = dict()
  df_dict[files[2][4:-4]] = 0
  for i in range(3, len(files)):
    res = face_classification(image1, f"{folderPath}/{folder}/{files[i]}")
    
    df_dict[files[i][4:-4]] = res['distance']

    print(f"Age 1st:{files[2][4:-4]} Age 2nd: {files[i][4:-4]}, verified: {res['verified']}, cosine distance: {res['distance']}")

  print(df_dict)
  main_dic[files[2][:3]] = df_dict
  print(main_dic)
  print()
  print()

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 31ms/step
Age 1st:07 Age 2nd: 15, verified: True, cosine distance: 0.1634006123054067
1/1 [==============================] - 0s 28ms/step
Age 1st:07 Age 2nd: 17, verified: True, cosine distance: 0.23424704291280285
1/1 [==============================] - 0s 27ms/step
Age 1st:07 Age 2nd: 18, verified: True, cosine distance: 0.2646065664368833
1/1 [==============================] - 0s 28ms/step
Age 1st:07 Age 2nd: 19, verified: True, cosine distance: 0.3217378031768474
1/1 [==============================] - 0s 27ms/step
Age 1st:07 Age 2nd: 25, verified: True, cosine distance: 0.2823379232949018
1/1 [==============================] - 0s 27ms/step
Age 1st:07 Age 2nd: 26, verified: True, cosine distance: 0.2613172946460064
1/1 [==============================] - 0s 44ms/step
Age 1st:07 Age 2nd: 29, verified: True, cosine distance: 0.26230307898994365
1/1 [==============================] - 0s 29ms/step

In [7]:
print(main_dic)

{'001': {'08': 0, '10': 0.15917173212387092, '14': 0.322313456466781, '16': 0.29240770392094173, '18': 0.44541380364631633, '19': 0.40452902921426004, '22': 0.34814981524053923, '28': 0.2721238628435688, '29': 0.3088517471219622, '33': 0.38429077107587073, '40': 0.46335924627638125, '43a': 0.47507719728820474, '43b': 0.4524556241291585}, '002': {'05': 0, '07': 0.13933748312851424, '12': 0.2500832156703908, '15': 0.30685683323345514, '16': 0.3147819416585621, '18': 0.3233150929639237, '20': 0.22434840012469293, '21': 0.4046713617478209, '23': 0.47671376293289225, '26': 0.4200054471801621, '29': 0.3635863544722786, '31': 0.3657926931859573, '36': 0.3140313683513607, '38': 0.3370174192553185}, '003': {'23': 0, '25': 0.1791624690157928, '35': 0.21408911970026467, '38': 0.2507973043858095, '47': 0.30428476676942673, '49': 0.24984161522519432, '51': 0.34749695323305974, '58': 0.3969750965176353, '60': 0.3552138206306782, '61': 0.34584603437097017}, '004': {'26': 0, '28': 0.2590506549679127, 

In [8]:
df =  pd.DataFrame(main_dic)
df = df.sort_index()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [9]:
df

,001,002,003,004,005,006,007,008,009,010,...,073,074,075,076,077,078,079,080,081,082
02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
02b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,0.346384,0.394207,0.268613,0.302375,0.421975,0.144695,0.232698,0.263958,0.223865,NaN
03a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,NaN,NaN,0.345846,NaN,0.383738,0.118720,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,0.411193,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,0.358646,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN,0.292672,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.to_csv('drive/MyDrive/Major proj Colab notebooks/thresholdValues.csv')

In [11]:
pd.read_csv('drive/MyDrive/Major proj Colab notebooks/thresholdValues.csv', index_col=0)

,001,002,003,004,005,006,007,008,009,010,...,073,074,075,076,077,078,079,080,081,082
02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
02b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,0.346384,0.394207,0.268613,0.302375,0.421975,0.144695,0.232698,0.263958,0.223865,NaN
03a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,NaN,NaN,0.345846,NaN,0.383738,0.118720,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,0.411193,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,0.358646,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN,0.292672,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
